<a href="https://colab.research.google.com/github/rahulsm27/ML/blob/main/Text_Generation_using_Fnet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 1. Install libraries and import

In [1]:
!pip install datasets
!pip install torch[transformers]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 11.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 17.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 18.5 MB/s eta 0:00:00


In [2]:
import torch

In [3]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

cuda


## 2. Loading Data

In [4]:
from datasets import load_dataset
datasets = load_dataset('wikitext','wikitext-2-raw-v1')

Generating test split:   0%|          | 0/4358 [00:00<?, ? examples/s]

Generating train split:   0%|          | 0/36718 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/3760 [00:00<?, ? examples/s]

In [5]:
datasets

DatasetDict({
    test: Dataset({
        features: ['text'],
        num_rows: 4358
    })
    train: Dataset({
        features: ['text'],
        num_rows: 36718
    })
    validation: Dataset({
        features: ['text'],
        num_rows: 3760
    })
})

## 3. PREPROCESSING DATA

In [6]:
import re
def preprocess_text(sentence):
  text = sentence['text'].lower() # lowering the sentence and storing in text vaiable
  text = re.sub('[^a-z?!.,]',' ',text) # removing other than characters and punctuations
  text = re.sub('\s\s+',' ',text) # removing double spaces
  sentence['text'] = text
  return sentence

In [7]:
datasets['train'] = datasets['train'].map(preprocess_text)
datasets['test'] = datasets['test'].map(preprocess_text)
datasets['validation'] = datasets['validation'].map(preprocess_text)

Map:   0%|          | 0/36718 [00:00<?, ? examples/s]

Map:   0%|          | 0/4358 [00:00<?, ? examples/s]

Map:   0%|          | 0/3760 [00:00<?, ? examples/s]

In [8]:
datasets['train'] = datasets['train'].filter(lambda x : len(x['text']) > 20)
datasets['test'] = datasets['test'].filter(lambda x : len(x['text']) > 20)
datasets['validation'] = datasets['validation'].filter(lambda x : len(x['text']) > 20)

Filter:   0%|          | 0/36718 [00:00<?, ? examples/s]

Filter:   0%|          | 0/4358 [00:00<?, ? examples/s]

Filter:   0%|          | 0/3760 [00:00<?, ? examples/s]

In [9]:
datasets

DatasetDict({
    test: Dataset({
        features: ['text'],
        num_rows: 2312
    })
    train: Dataset({
        features: ['text'],
        num_rows: 18794
    })
    validation: Dataset({
        features: ['text'],
        num_rows: 1988
    })
})

## 3. TOKENIZATION

In [10]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')

tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [11]:
tokenizer.special_tokens_map

{'unk_token': '[UNK]',
 'sep_token': '[SEP]',
 'pad_token': '[PAD]',
 'cls_token': '[CLS]',
 'mask_token': '[MASK]'}

In [12]:
vocab_size = len(tokenizer.vocab)

In [13]:
def tokenize(sentence):
  sentence = tokenizer(sentence['text'],truncation = True)

  return sentence

tokenized_inputs = datasets['train'].map(tokenize)

Map:   0%|          | 0/18794 [00:00<?, ? examples/s]

In [14]:
from transformers import DataCollatorWithPadding
from torch.utils.data import DataLoader

batch = 16

data_collator = DataCollatorWithPadding(tokenizer=tokenizer, padding=True)
dataloader = DataLoader(list(zip(tokenized_inputs['input_ids'], tokenized_inputs['attention_mask'])),batch_size=batch,
                        collate_fn=data_collator)

## 4. MODEL

In [15]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.fft as fft

In [16]:
class FNetEncoder(nn.Module):

  def __init__(self,embed_dim, dense_dim):
    super(FNetEncoder,self).__init__()
    self.embed_dim = embed_dim
    self.dense_dim = dense_dim
#    self.num_heads = num_heads


   # self.positional_embedding = nn.Embedding(max_seq_len, embed_dim)
    #self.embedding = nn.Embedding(vocab_size,embed_dim)
    self.dense_proj = nn.Sequential(nn.Linear(self.embed_dim,self.dense_dim), nn.ReLU(), nn.Linear(self.dense_dim,self.embed_dim))

    self.layernorm_1 = nn.LayerNorm(self.embed_dim)
    self.layernorm_2 = nn.LayerNorm(self.embed_dim)

  def forward(self,inputs):
    # embeddign the inputs
    #embedded_inputs = self.embedding(inputs)

    # # Adding positional embeddings
    # seq_length = inputs.size(1)
    # positions = torch.arange(0, seq_length, device=inputs.device).unsqueeze(0)
    # positional_embeddings = self.positional_embedding(positions)

    # # Summing token embeddings and positional embeddings
    # embedded_inputs = embedded_inputs + positional_embeddings

    #castinn the inputs to compex
   # inp_complex = torch.view_as_complex(inputs)

    # getting fourier transform
    fft_result = fft.fft2(inputs)

    #taking real part
    fft_real = fft_result.real.float()

    proj_input = self.layernorm_1 (inputs + fft_real)
    proj_output = self.dense_proj(proj_input)
    return self.layernorm_2(proj_input +proj_output)





In [63]:
class FNetDecoder(nn.Module):

  def __init__(self,embed_dim,dense_dim,num_heads):
    super(FNetDecoder,self).__init__()
    self.embed_dim = embed_dim
    self.dense_dim = dense_dim
    self.num_heads = num_heads

    self.attention_1 = nn.MultiheadAttention(embed_dim,num_heads)
    self.attention_2 = nn.MultiheadAttention(embed_dim,num_heads)

    self.dense_proj = nn.Sequential(nn.Linear(embed_dim, latent_dim),nn.ReLU(),nn.Linear(latent_dim, embed_dim))

    self.layernorm_1 = nn.LayerNorm(embed_dim)
    self.layernorm_2 = nn.LayerNorm(embed_dim)
    self.layernorm_3 = nn.LayerNorm(embed_dim)

  def forward(self, inputs, encoder_outputs, mask=None):
  #  causal_mask = self.get_causal_attention_mask(inputs)
    causal_mask = nn.Transformer.generate_square_subsequent_mask(inputs.size(0)).to(device)

    if mask is not None:
        padding_mask = mask.unsqueeze(1)
        padding_mask = torch.min(padding_mask, causal_mask)

    attention_output_1, _ = self.attention_1(inputs, inputs, inputs, attn_mask=causal_mask)
    out_1 = self.layernorm_1(inputs + attention_output_1)

    attention_output_2, _ = self.attention_2(out_1, encoder_outputs, encoder_outputs, attn_mask=causal_mask)
    out_2 = self.layernorm_2(out_1 + attention_output_2)

    proj_output = self.dense_proj(out_2)
    return self.layernorm_3(out_2 + proj_output)

  def get_causal_attention_mask(self, inputs):
    # batch_size, sequence_length = inputs.size(0), inputs.size(1)
    # i = torch.arange(sequence_length).unsqueeze(0).expand(batch_size, -1)
    # j = torch.arange(sequence_length).unsqueeze(1).expand(-1, sequence_length)
    # mask = i >= j
    # mask = mask.unsqueeze(0)
    # return mask

    # input_shape = inputs.size()
    # batch_size, sequence_length = input_shape[0], input_shape[1]
    # causal_mask = torch.triu(torch.ones((sequence_length, sequence_length), device=inputs.device), diagonal=1)
    # causal_mask = causal_mask.unsqueeze(0) # Add batch and head dimensions
    # return causal_mask

      # Create a tensor of ones with the shape (length, length).
    input_shape = inputs.size()
    length = input_shape[1]
    mask = torch.ones((16, 16), dtype=torch.bool, device=torch.device("cuda" if torch.cuda.is_available() else "cpu"))
  # Create a diagonal matrix of zeros with the shape (length, length).
    diag = torch.zeros((16, 16), dtype=torch.bool, device=torch.device("cuda" if torch.cuda.is_available() else "cpu"))
  # Set the diagonal elements of the mask to True.
    mask = mask.masked_fill(diag, True)
  # Return the mask.
    return mask

In [75]:

class PositionalEmbedding(nn.Module):
  def __init__(self, sequence_length, vocab_size, embed_dim):
    super(PositionalEmbedding, self).__init__()
    self.token_embeddings = nn.Embedding(vocab_size, embed_dim)
    self.position_embeddings = nn.Embedding(sequence_length, embed_dim)

  def forward(self, inputs):
    length = inputs.size(-1)
    positions = torch.arange(0, length, device=inputs.device).unsqueeze(0)
    embedded_tokens = self.token_embeddings(inputs)
    embedded_positions = self.position_embeddings(positions)
    return embedded_tokens + embedded_positions



class FNetTextGenerator(nn.Module):
  def __init__(self, embed_dim, latent_dim, vocab_size, max_seq_len, num_heads):
    super(FNetTextGenerator, self).__init__()
    self.positional = PositionalEmbedding(max_seq_len, vocab_size, embed_dim)
    self.encoder = FNetEncoder(embed_dim,latent_dim)
    self.decoder = FNetDecoder(embed_dim, latent_dim, num_heads)

  def forward(self, inputs, target=None):
    positional_inputs = self.positional(inputs)
    encoder_output = self.encoder(positional_inputs)
#    print(encoder_output.size())
    if target is not None:
      decoder_output = self.decoder(target, encoder_output)
      return decoder_output

    # If no target is provided, generate autoregressively
    batch_size, seq_len = inputs.size()
  #  print(inputs.size())
    generated_sequence = torch.zeros(batch_size, seq_len, dtype=torch.long, device=inputs.device)

        # Initial input for autoregressive decoding
    input_token = inputs[:, 0].unsqueeze(1).float()
 #   print(input_token)

# ...

    for t in range(1, seq_len):
      decoder_output = self.decoder(positional_inputs, encoder_output)

    # Use torch.argmax directly to get the indices of the maximum values
      predicted_token = torch.argmax(F.softmax(decoder_output, dim=-1), dim=-1)

    # Ensure that predicted_token has the correct shape
      predicted_token = predicted_token.unsqueeze(1)

    # Update the generated sequence
      generated_sequence[:, t] = predicted_token.squeeze()

    # Update input_token for the next iteration
      input_token = predicted_token



    return generated_sequence

# Example usage
embed_dim = 256
latent_dim = 512
vocab_size = 10000
max_seq_len = 50
num_heads = 8



model = FNetTextGenerator(embed_dim, latent_dim, vocab_size, max_seq_len, num_heads).to(device)

# Dummy input
inputs = torch.randint(0, vocab_size, (16, max_seq_len)).long().to(device)


# Forward pass for text generation
generated_sequence = model(inputs)
print("Generated Sequence:", generated_sequence)

RuntimeError: ignored

In [30]:
inputs = torch.randint(0, vocab_size, (16, max_seq_len)).to(device)

In [25]:
inputs.dtype

torch.int64

In [50]:
class FNetTextGenerator(nn.Module):
    def __init__(self, embed_dim, latent_dim, vocab_size, max_seq_len, num_heads, num_layers):
        super(FNetTextGenerator, self).__init__()

        self.embedding = nn.Embedding(vocab_size, embed_dim)
        self.positional_embedding = nn.Embedding(max_seq_len, embed_dim)
        self.transformer_decoder_layer = nn.TransformerDecoderLayer(embed_dim, num_heads)
        self.transformer_decoder = nn.TransformerDecoder(self.transformer_decoder_layer, num_layers=num_layers)

        self.transformer_encoder_layer = FNetEncoder(embed_dim, latent_dim)
      #  self.transformer_encoder = torch.nn.TransformerEncoder(self.transformer_encoder_layer, num_layers = num_layers)

        self.fc = nn.Linear(embed_dim, vocab_size)

    def forward(self, inputs, target=None):
        seq_length = inputs.size(1)

        embedded_tokens = self.embedding(inputs)
        positions = torch.arange(0, seq_length, device=inputs.device).unsqueeze(0)
        embedded_positions = self.positional_embedding(positions)
        encoder_input = embedded_tokens + embedded_positions

        encoder_output = self.transformer_encoder_layer(encoder_input)

        if target is not None:
            target = target.permute(1, 0, 2)  # Adjust the shape for transformer decoder
            decoder_output = self.transformer_decoder(target, encoder_output)
            return decoder_output.permute(1, 0, 2)  # Adjust the shape back to batch-first

        # If no target is provided, generate autoregressively
        batch_size, seq_len = inputs.size()
        generated_sequence = torch.zeros(batch_size, seq_len, dtype=torch.long, device=inputs.device)

        # Initial input for autoregressive decoding
        input_token = inputs[:, 0].unsqueeze(1)

        for t in range(1, seq_len):
            embedded_input_token = self.embedding(input_token) + self.positional_embedding(torch.tensor([[t]]).to(inputs.device))
            decoder_output = self.transformer_decoder(embedded_input_token, encoder_output)
            logits = self.fc(decoder_output[-1])
            _, predicted_token = torch.max(F.softmax(logits, dim=-1), dim=-1)
            generated_sequence[:, t] = predicted_token.squeeze()

            input_token = predicted_token.unsqueeze(1)

        return generated_sequence



In [61]:
# Example usage
embed_dim = 256
latent_dim = 512
vocab_size = 10000
max_seq_len = 50
num_heads = 8
num_layers = 6

model = FNetTextGenerator(embed_dim, latent_dim, vocab_size, max_seq_len, num_heads, num_layers).to(device)

# Dummy input
inputs = torch.randint(0, vocab_size, (2, max_seq_len)).to(device)

# Forward pass for text generation
generated_sequence = model(inputs)
print("Generated Sequence:", generated_sequence)

TypeError: ignored